In [1]:
%%bash
pip install --upgrade duckdb
pip install --upgrade jupysql
pip install --upgrade duckdb-engine

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for duckdb from https://files.pythonhosted.org/packages/b1/a1/92388a93c4f6b310219ef02e192b1e5c8792c5becc136de68b0256364de8/duckdb-0.9.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 34.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: duckdb
    Found existing installation: duckdb 0.8.1
    Uninstalling duckdb-0.8.1:
      Successfully uninstalled duckdb-0.8.1
Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for jupysql from https://files.pythonhosted.org/packages/34/ca/4ea8ba339edb13f3cffcf76390b4f35f5f0478c994f254ccf1ae49104241/jupysql-0.10.2-py3-none-any.whl.metadata
  Obtaining dependency information for jupysql-plugin from https://files.pythonhosted.org/packages/8d/23/005473708375e9e17c5913c6a6a1208bd4d2f20e4dccc98093fb8ec124d

In [10]:
cd "/home/marchesini_id/buckets/b1/datasets"

/home/marchesini_id/buckets/b1/datasets


In [12]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///dmeyf.db


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [13]:
%%sql
create or replace table competencia_02 as
select
    *
from read_csv_auto('competencia_02_ternaria.csv.gz')


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [14]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
from competencia_02
limit 10

,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter
0,29188883,201901,43,NaN
1,29188883,201902,45,43.0
2,29188883,201903,39,45.0
3,29188883,201904,38,39.0
4,29188883,201905,39,38.0
5,29188883,201906,41,39.0
6,29188883,201907,34,41.0
7,29188883,201908,30,34.0
8,29188883,201909,25,30.0
9,29188883,201910,30,25.0


In [15]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
  , ctrx_quarter - lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
from competencia_02
limit 10

,numero_de_cliente,foto_mes,ctrx_quarter,lag_1_ctrx_quarter,delta_1_ctrx_quarter,lag_2_ctrx_quarter
0,29188883,201901,43,NaN,NaN,NaN
1,29188883,201902,45,43.0,2.0,NaN
2,29188883,201903,39,45.0,-6.0,-4.0
3,29188883,201904,38,39.0,-1.0,-7.0
4,29188883,201905,39,38.0,1.0,0.0
5,29188883,201906,41,39.0,2.0,3.0
6,29188883,201907,34,41.0,-7.0,-5.0
7,29188883,201908,30,34.0,-4.0,-11.0
8,29188883,201909,25,30.0,-5.0,-9.0
9,29188883,201910,30,25.0,5.0,0.0


In [16]:

%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , avg(ctrx_quarter) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 6 preceding and current row) as avg_6_ctrx_quarter
from competencia_02
limit 10

,numero_de_cliente,foto_mes,ctrx_quarter,avg_6_ctrx_quarter
0,29188883,201901,43,43.000000
1,29188883,201902,45,44.000000
2,29188883,201903,39,42.333333
3,29188883,201904,38,41.250000
4,29188883,201905,39,40.800000
5,29188883,201906,41,40.833333
6,29188883,201907,34,39.857143
7,29188883,201908,30,38.000000
8,29188883,201909,25,35.142857
9,29188883,201910,30,33.857143


In [17]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , avg(ctrx_quarter) over ventana_6 as ctrx_quarter_media_6
  , max(ctrx_quarter) over ventana_6 as ctrx_quarter_max_6
  , min(ctrx_quarter) over ventana_6 as ctrx_quarter_min_6
from competencia_02
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row)
limit 10

,numero_de_cliente,foto_mes,ctrx_quarter,ctrx_quarter_media_6,ctrx_quarter_max_6,ctrx_quarter_min_6
0,29188883,201901,43,43.000000,43,43
1,29188883,201902,45,44.000000,45,43
2,29188883,201903,39,42.333333,45,39
3,29188883,201904,38,41.250000,45,38
4,29188883,201905,39,40.800000,45,38
5,29188883,201906,41,40.833333,45,38
6,29188883,201907,34,39.857143,45,34
7,29188883,201908,30,38.000000,45,30
8,29188883,201909,25,35.142857,41,25
9,29188883,201910,30,33.857143,41,25


In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_quarter_slope_6
from competencia_02
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row)
limit 10

In [32]:
campos = ['mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 'mcuenta_corriente_adicional', 'mcuenta_corriente', 'mcaja_ahorro', 'mcaja_ahorro_adicional', 'mcaja_ahorro_dolares', 'mcuentas_saldo', 'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'ctarjeta_master_transacciones', 'mtarjeta_master_consumo', 'mprestamos_personales', 'mprestamos_prendarios', 'mprestamos_hipotecarios', 'mplazo_fijo_dolares', 'mplazo_fijo_pesos', 'minversion1_pesos', 'minversion1_dolares', 'minversion2', 'cpayroll_trx', 'mpayroll', 'mpayroll2', 'cpayroll2_trx', 'mcuenta_debitos_automaticos', 'mttarjeta_visa_debitos_automaticos', 'mttarjeta_master_debitos_automaticos', 'mpagodeservicios', 'mpagomiscuentas', 'mcajeros_propios_descuentos', 'mtarjeta_visa_descuentos', 'mtarjeta_master_descuentos', 'mcomisiones_mantenimiento', 'mcomisiones_otras', 'mforex_buy', 'mforex_sell', 'mtransferencias_recibidas', 'mtransferencias_emitidas', 'mextraccion_autoservicio', 'mcheques_depositados', 'mcheques_emitidos', 'mcheques_depositados_rechazados', 'mcheques_emitidos_rechazados', 'matm',  'matm_other', 'ctrx_quarter', 'Master_msaldototal', 'Master_msaldopesos', 'Master_msaldodolares', 'Master_mconsumospesos', 'Master_mconsumosdolares', 'Master_mlimitecompra', 'Master_madelantopesos', 'Master_madelantodolares', 'Master_mpagado', 'Master_mpagospesos', 'Master_mpagosdolares', 'Master_mconsumototal', 'Master_cconsumos', 'Master_cadelantosefectivo', 'Master_mpagominimo',  'Visa_msaldototal', 'Visa_msaldopesos', 'Visa_msaldodolares', 'Visa_mconsumospesos', 'Visa_mconsumosdolares', 'Visa_mlimitecompra', 'Visa_madelantopesos', 'Visa_madelantodolares', 'Visa_mpagado', 'Visa_mpagospesos', 'Visa_mpagosdolares', 'Visa_mconsumototal', 'Visa_cconsumos', 'Visa_cadelantosefectivo', 'Visa_mpagominimo']
nuevos_features_regr = ""
nuevos_features_lag =""
for campo in campos:
  nuevos_features_regr += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_6 as ctrx_{campo}_slope_6"
  nuevos_features_lag += f"\n, lag({campo}, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_{campo}"
print(nuevos_features_regr)
print(nuevos_features_lag)



, regr_slope(mrentabilidad, cliente_antiguedad) over ventana_6 as ctrx_mrentabilidad_slope_6
, regr_slope(mrentabilidad_annual, cliente_antiguedad) over ventana_6 as ctrx_mrentabilidad_annual_slope_6
, regr_slope(mcomisiones, cliente_antiguedad) over ventana_6 as ctrx_mcomisiones_slope_6
, regr_slope(mactivos_margen, cliente_antiguedad) over ventana_6 as ctrx_mactivos_margen_slope_6
, regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_6 as ctrx_mpasivos_margen_slope_6
, regr_slope(mcuenta_corriente_adicional, cliente_antiguedad) over ventana_6 as ctrx_mcuenta_corriente_adicional_slope_6
, regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_6 as ctrx_mcuenta_corriente_slope_6
, regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_6 as ctrx_mcaja_ahorro_slope_6
, regr_slope(mcaja_ahorro_adicional, cliente_antiguedad) over ventana_6 as ctrx_mcaja_ahorro_adicional_slope_6
, regr_slope(mcaja_ahorro_dolares, cliente_antiguedad) over ventana_6 as ctrx_mcaja_ahorro_d

In [ ]:
%%sql
select * from competencia_02 limit 10

In [ ]:
%%sql
select cproductos, tcuentas, ccuenta_corriente, mpasivos_margen, mcuenta_corriente_adicional, mcuenta_corriente, ccaja_ahorro, mcaja_ahorro, mcaja_ahorro_adicional, mcaja_ahorro_dolares, cdescubierto_preacordado, mcuentas_saldo, ctarjeta_debito, ctarjeta_debito_transacciones from competencia_02 limit 10 

In [33]:
%%sql
select numero_de_cliente, foto_mes, ctrx_quarter
, regr_slope(mrentabilidad, cliente_antiguedad) over ventana_6 as ctrx_mrentabilidad_slope_6
, regr_slope(mrentabilidad_annual, cliente_antiguedad) over ventana_6 as ctrx_mrentabilidad_annual_slope_6
, regr_slope(mcomisiones, cliente_antiguedad) over ventana_6 as ctrx_mcomisiones_slope_6
, regr_slope(mactivos_margen, cliente_antiguedad) over ventana_6 as ctrx_mactivos_margen_slope_6
, regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_6 as ctrx_mpasivos_margen_slope_6
, regr_slope(mcuenta_corriente_adicional, cliente_antiguedad) over ventana_6 as ctrx_mcuenta_corriente_adicional_slope_6
, regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_6 as ctrx_mcuenta_corriente_slope_6
, regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_6 as ctrx_mcaja_ahorro_slope_6
, regr_slope(mcaja_ahorro_adicional, cliente_antiguedad) over ventana_6 as ctrx_mcaja_ahorro_adicional_slope_6
, regr_slope(mcaja_ahorro_dolares, cliente_antiguedad) over ventana_6 as ctrx_mcaja_ahorro_dolares_slope_6
, regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_6 as ctrx_mcuentas_saldo_slope_6
, regr_slope(ctarjeta_debito_transacciones, cliente_antiguedad) over ventana_6 as ctrx_ctarjeta_debito_transacciones_slope_6
, regr_slope(mautoservicio, cliente_antiguedad) over ventana_6 as ctrx_mautoservicio_slope_6
, regr_slope(ctarjeta_visa_transacciones, cliente_antiguedad) over ventana_6 as ctrx_ctarjeta_visa_transacciones_slope_6
, regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) over ventana_6 as ctrx_mtarjeta_visa_consumo_slope_6
, regr_slope(ctarjeta_master_transacciones, cliente_antiguedad) over ventana_6 as ctrx_ctarjeta_master_transacciones_slope_6
, regr_slope(mtarjeta_master_consumo, cliente_antiguedad) over ventana_6 as ctrx_mtarjeta_master_consumo_slope_6
, regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_6 as ctrx_mprestamos_personales_slope_6
, regr_slope(mprestamos_prendarios, cliente_antiguedad) over ventana_6 as ctrx_mprestamos_prendarios_slope_6
, regr_slope(mprestamos_hipotecarios, cliente_antiguedad) over ventana_6 as ctrx_mprestamos_hipotecarios_slope_6
, regr_slope(mplazo_fijo_dolares, cliente_antiguedad) over ventana_6 as ctrx_mplazo_fijo_dolares_slope_6
, regr_slope(mplazo_fijo_pesos, cliente_antiguedad) over ventana_6 as ctrx_mplazo_fijo_pesos_slope_6
, regr_slope(minversion1_pesos, cliente_antiguedad) over ventana_6 as ctrx_minversion1_pesos_slope_6
, regr_slope(minversion1_dolares, cliente_antiguedad) over ventana_6 as ctrx_minversion1_dolares_slope_6
, regr_slope(minversion2, cliente_antiguedad) over ventana_6 as ctrx_minversion2_slope_6
, regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_6 as ctrx_cpayroll_trx_slope_6
, regr_slope(mpayroll, cliente_antiguedad) over ventana_6 as ctrx_mpayroll_slope_6
, regr_slope(mpayroll2, cliente_antiguedad) over ventana_6 as ctrx_mpayroll2_slope_6
, regr_slope(cpayroll2_trx, cliente_antiguedad) over ventana_6 as ctrx_cpayroll2_trx_slope_6
, regr_slope(mcuenta_debitos_automaticos, cliente_antiguedad) over ventana_6 as ctrx_mcuenta_debitos_automaticos_slope_6
, regr_slope(mttarjeta_visa_debitos_automaticos, cliente_antiguedad) over ventana_6 as ctrx_mttarjeta_visa_debitos_automaticos_slope_6
, regr_slope(mttarjeta_master_debitos_automaticos, cliente_antiguedad) over ventana_6 as ctrx_mttarjeta_master_debitos_automaticos_slope_6
, regr_slope(mpagodeservicios, cliente_antiguedad) over ventana_6 as ctrx_mpagodeservicios_slope_6
, regr_slope(mpagomiscuentas, cliente_antiguedad) over ventana_6 as ctrx_mpagomiscuentas_slope_6
, regr_slope(mcajeros_propios_descuentos, cliente_antiguedad) over ventana_6 as ctrx_mcajeros_propios_descuentos_slope_6
, regr_slope(mtarjeta_visa_descuentos, cliente_antiguedad) over ventana_6 as ctrx_mtarjeta_visa_descuentos_slope_6
, regr_slope(mtarjeta_master_descuentos, cliente_antiguedad) over ventana_6 as ctrx_mtarjeta_master_descuentos_slope_6
, regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_6 as ctrx_mcomisiones_mantenimiento_slope_6
, regr_slope(mcomisiones_otras, cliente_antiguedad) over ventana_6 as ctrx_mcomisiones_otras_slope_6
, regr_slope(mforex_buy, cliente_antiguedad) over ventana_6 as ctrx_mforex_buy_slope_6
, regr_slope(mforex_sell, cliente_antiguedad) over ventana_6 as ctrx_mforex_sell_slope_6
, regr_slope(mtransferencias_recibidas, cliente_antiguedad) over ventana_6 as ctrx_mtransferencias_recibidas_slope_6
, regr_slope(mtransferencias_emitidas, cliente_antiguedad) over ventana_6 as ctrx_mtransferencias_emitidas_slope_6
, regr_slope(mextraccion_autoservicio, cliente_antiguedad) over ventana_6 as ctrx_mextraccion_autoservicio_slope_6
, regr_slope(mcheques_depositados, cliente_antiguedad) over ventana_6 as ctrx_mcheques_depositados_slope_6
, regr_slope(mcheques_emitidos, cliente_antiguedad) over ventana_6 as ctrx_mcheques_emitidos_slope_6
, regr_slope(mcheques_depositados_rechazados, cliente_antiguedad) over ventana_6 as ctrx_mcheques_depositados_rechazados_slope_6
, regr_slope(mcheques_emitidos_rechazados, cliente_antiguedad) over ventana_6 as ctrx_mcheques_emitidos_rechazados_slope_6
, regr_slope(matm, cliente_antiguedad) over ventana_6 as ctrx_matm_slope_6
, regr_slope(matm_other, cliente_antiguedad) over ventana_6 as ctrx_matm_other_slope_6
, regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_ctrx_quarter_slope_6
, regr_slope(Master_msaldototal, cliente_antiguedad) over ventana_6 as ctrx_Master_msaldototal_slope_6
, regr_slope(Master_msaldopesos, cliente_antiguedad) over ventana_6 as ctrx_Master_msaldopesos_slope_6
, regr_slope(Master_msaldodolares, cliente_antiguedad) over ventana_6 as ctrx_Master_msaldodolares_slope_6
, regr_slope(Master_mconsumospesos, cliente_antiguedad) over ventana_6 as ctrx_Master_mconsumospesos_slope_6
, regr_slope(Master_mconsumosdolares, cliente_antiguedad) over ventana_6 as ctrx_Master_mconsumosdolares_slope_6
, regr_slope(Master_mlimitecompra, cliente_antiguedad) over ventana_6 as ctrx_Master_mlimitecompra_slope_6
, regr_slope(Master_madelantopesos, cliente_antiguedad) over ventana_6 as ctrx_Master_madelantopesos_slope_6
, regr_slope(Master_madelantodolares, cliente_antiguedad) over ventana_6 as ctrx_Master_madelantodolares_slope_6
, regr_slope(Master_mpagado, cliente_antiguedad) over ventana_6 as ctrx_Master_mpagado_slope_6
, regr_slope(Master_mpagospesos, cliente_antiguedad) over ventana_6 as ctrx_Master_mpagospesos_slope_6
, regr_slope(Master_mpagosdolares, cliente_antiguedad) over ventana_6 as ctrx_Master_mpagosdolares_slope_6
, regr_slope(Master_mconsumototal, cliente_antiguedad) over ventana_6 as ctrx_Master_mconsumototal_slope_6
, regr_slope(Master_cconsumos, cliente_antiguedad) over ventana_6 as ctrx_Master_cconsumos_slope_6
, regr_slope(Master_cadelantosefectivo, cliente_antiguedad) over ventana_6 as ctrx_Master_cadelantosefectivo_slope_6
, regr_slope(Master_mpagominimo, cliente_antiguedad) over ventana_6 as ctrx_Master_mpagominimo_slope_6
, regr_slope(Visa_msaldototal, cliente_antiguedad) over ventana_6 as ctrx_Visa_msaldototal_slope_6
, regr_slope(Visa_msaldopesos, cliente_antiguedad) over ventana_6 as ctrx_Visa_msaldopesos_slope_6
, regr_slope(Visa_msaldodolares, cliente_antiguedad) over ventana_6 as ctrx_Visa_msaldodolares_slope_6
, regr_slope(Visa_mconsumospesos, cliente_antiguedad) over ventana_6 as ctrx_Visa_mconsumospesos_slope_6
, regr_slope(Visa_mconsumosdolares, cliente_antiguedad) over ventana_6 as ctrx_Visa_mconsumosdolares_slope_6
, regr_slope(Visa_mlimitecompra, cliente_antiguedad) over ventana_6 as ctrx_Visa_mlimitecompra_slope_6
, regr_slope(Visa_madelantopesos, cliente_antiguedad) over ventana_6 as ctrx_Visa_madelantopesos_slope_6
, regr_slope(Visa_madelantodolares, cliente_antiguedad) over ventana_6 as ctrx_Visa_madelantodolares_slope_6
, regr_slope(Visa_mpagado, cliente_antiguedad) over ventana_6 as ctrx_Visa_mpagado_slope_6
, regr_slope(Visa_mpagospesos, cliente_antiguedad) over ventana_6 as ctrx_Visa_mpagospesos_slope_6
, regr_slope(Visa_mpagosdolares, cliente_antiguedad) over ventana_6 as ctrx_Visa_mpagosdolares_slope_6
, regr_slope(Visa_mconsumototal, cliente_antiguedad) over ventana_6 as ctrx_Visa_mconsumototal_slope_6
, regr_slope(Visa_cconsumos, cliente_antiguedad) over ventana_6 as ctrx_Visa_cconsumos_slope_6
, regr_slope(Visa_cadelantosefectivo, cliente_antiguedad) over ventana_6 as ctrx_Visa_cadelantosefectivo_slope_6
, regr_slope(Visa_mpagominimo, cliente_antiguedad) over ventana_6 as ctrx_Visa_mpagominimo_slope_6
, lag(mrentabilidad, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mrentabilidad
, lag(mrentabilidad_annual, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mrentabilidad_annual
, lag(mcomisiones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones
, lag(mactivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mactivos_margen
, lag(mpasivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpasivos_margen
, lag(mcuenta_corriente_adicional, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_corriente_adicional
, lag(mcuenta_corriente, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_corriente
, lag(mcaja_ahorro, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro
, lag(mcaja_ahorro_adicional, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro_adicional
, lag(mcaja_ahorro_dolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro_dolares
, lag(mcuentas_saldo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuentas_saldo
, lag(ctarjeta_debito_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_debito_transacciones
, lag(mautoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mautoservicio
, lag(ctarjeta_visa_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa_transacciones
, lag(mtarjeta_visa_consumo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_visa_consumo
, lag(ctarjeta_master_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_master_transacciones
, lag(mtarjeta_master_consumo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_master_consumo
, lag(mprestamos_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_personales
, lag(mprestamos_prendarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_prendarios
, lag(mprestamos_hipotecarios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_hipotecarios
, lag(mplazo_fijo_dolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mplazo_fijo_dolares
, lag(mplazo_fijo_pesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mplazo_fijo_pesos
, lag(minversion1_pesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion1_pesos
, lag(minversion1_dolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion1_dolares
, lag(minversion2, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_minversion2
, lag(cpayroll_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll_trx
, lag(mpayroll, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll
, lag(mpayroll2, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll2
, lag(cpayroll2_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll2_trx
, lag(mcuenta_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_debitos_automaticos
, lag(mttarjeta_visa_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mttarjeta_visa_debitos_automaticos
, lag(mttarjeta_master_debitos_automaticos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mttarjeta_master_debitos_automaticos
, lag(mpagodeservicios, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpagodeservicios
, lag(mpagomiscuentas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpagomiscuentas
, lag(mcajeros_propios_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcajeros_propios_descuentos
, lag(mtarjeta_visa_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_visa_descuentos
, lag(mtarjeta_master_descuentos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_master_descuentos
, lag(mcomisiones_mantenimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_mantenimiento
, lag(mcomisiones_otras, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_otras
, lag(mforex_buy, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mforex_buy
, lag(mforex_sell, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mforex_sell
, lag(mtransferencias_recibidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtransferencias_recibidas
, lag(mtransferencias_emitidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtransferencias_emitidas
, lag(mextraccion_autoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mextraccion_autoservicio
, lag(mcheques_depositados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_depositados
, lag(mcheques_emitidos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_emitidos
, lag(mcheques_depositados_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_depositados_rechazados
, lag(mcheques_emitidos_rechazados, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcheques_emitidos_rechazados
, lag(matm, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_matm
, lag(matm_other, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_matm_other
, lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
, lag(Master_msaldototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldototal
, lag(Master_msaldopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldopesos
, lag(Master_msaldodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_msaldodolares
, lag(Master_mconsumospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumospesos
, lag(Master_mconsumosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumosdolares
, lag(Master_mlimitecompra, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mlimitecompra
, lag(Master_madelantopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_madelantopesos
, lag(Master_madelantodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_madelantodolares
, lag(Master_mpagado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagado
, lag(Master_mpagospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagospesos
, lag(Master_mpagosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagosdolares
, lag(Master_mconsumototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mconsumototal
, lag(Master_cconsumos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_cconsumos
, lag(Master_cadelantosefectivo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_cadelantosefectivo
, lag(Master_mpagominimo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_mpagominimo
, lag(Visa_msaldototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldototal
, lag(Visa_msaldopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldopesos
, lag(Visa_msaldodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldodolares
, lag(Visa_mconsumospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumospesos
, lag(Visa_mconsumosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumosdolares
, lag(Visa_mlimitecompra, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mlimitecompra
, lag(Visa_madelantopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_madelantopesos
, lag(Visa_madelantodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_madelantodolares
, lag(Visa_mpagado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagado
, lag(Visa_mpagospesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagospesos
, lag(Visa_mpagosdolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagosdolares
, lag(Visa_mconsumototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mconsumototal
, lag(Visa_cconsumos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_cconsumos
, lag(Visa_cadelantosefectivo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_cadelantosefectivo
, lag(Visa_mpagominimo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagominimo
from competencia_02 
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row)
limit 10

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,numero_de_cliente,foto_mes,ctrx_quarter,ctrx_mrentabilidad_slope_6,ctrx_mrentabilidad_annual_slope_6,ctrx_mcomisiones_slope_6,ctrx_mactivos_margen_slope_6,ctrx_mpasivos_margen_slope_6,ctrx_mcuenta_corriente_adicional_slope_6,ctrx_mcuenta_corriente_slope_6,...,lag_1_Visa_mlimitecompra,lag_1_Visa_madelantopesos,lag_1_Visa_madelantodolares,lag_1_Visa_mpagado,lag_1_Visa_mpagospesos,lag_1_Visa_mpagosdolares,lag_1_Visa_mconsumototal,lag_1_Visa_cconsumos,lag_1_Visa_cadelantosefectivo,lag_1_Visa_mpagominimo
0,29188883,201901,43,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29188883,201902,45,984.710000,40616.290000,-2.540000,12.330000,829.320000,0.0,852.830000,...,462742.86,0.0,0.0,0.00,-1332.76,0.0,1102.50,1.0,0.0,58.65
2,29188883,201903,39,6839.150000,45565.940000,153.880000,7.905000,5691.390000,0.0,-315.530000,...,489096.94,0.0,0.0,0.00,-1122.56,0.0,1085.02,1.0,0.0,58.65
3,29188883,201904,38,4551.665000,47003.406000,115.681000,-2.964000,3784.710000,0.0,394.908000,...,507903.60,0.0,0.0,1346.93,-1103.79,0.0,1324.31,1.0,0.0,70.38
4,29188883,201905,39,-11976.787000,-82022.312000,-169.185000,0.729000,-10066.884000,0.0,544.980000,...,563040.00,0.0,0.0,0.00,-1346.93,0.0,1300.85,1.0,0.0,70.38
5,29188883,201906,41,-5027.476571,-9818.914000,-43.508857,-5.228571,-4243.677429,0.0,274.293429,...,563040.00,0.0,0.0,0.00,-1323.62,0.0,1364.19,1.0,0.0,70.38
6,29188883,201907,34,-2176.188214,20179.726071,9.443214,-1.864286,-1861.418571,0.0,-20.287500,...,563040.00,0.0,0.0,1620.81,-1388.06,0.0,1592.93,1.0,0.0,82.11
7,29188883,201908,30,-872.625714,39935.507857,52.995357,-5.845000,-783.260357,0.0,5.756071,...,563040.00,0.0,0.0,0.00,-1620.81,0.0,1364.19,1.0,0.0,70.38
8,29188883,201909,25,46.854286,57154.344643,81.653571,-7.236429,-22.430357,0.0,-68.477143,...,563040.00,0.0,0.0,1548.00,-1388.06,0.0,1521.38,1.0,0.0,82.11
9,29188883,201910,30,-3777.075714,-8301.591071,-23.033214,-0.862143,-3199.515357,0.0,-183.597857,...,563040.00,0.0,0.0,1712.70,-1548.00,0.0,1683.25,1.0,0.0,93.84


In [34]:
%%sql
create or replace table competencia_02 as
select *, regr_slope(mrentabilidad, cliente_antiguedad) over ventana_6 as ctrx_mrentabilidad_slope_6
, regr_slope(mrentabilidad_annual, cliente_antiguedad) over ventana_6 as ctrx_mrentabilidad_annual_slope_6
, regr_slope(mcomisiones, cliente_antiguedad) over ventana_6 as ctrx_mcomisiones_slope_6
, regr_slope(mactivos_margen, cliente_antiguedad) over ventana_6 as ctrx_mactivos_margen_slope_6
, regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_6 as ctrx_mpasivos_margen_slope_6
, regr_slope(cproductos, cliente_antiguedad) over ventana_6 as ctrx_cproductos_slope_6
, regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_6 as ctrx_mcuenta_corriente_slope_6
, regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_6 as ctrx_mcaja_ahorro_slope_6
, regr_slope(mcaja_ahorro_adicional, cliente_antiguedad) over ventana_6 as ctrx_mcaja_ahorro_adicional_slope_6
, regr_slope(mcaja_ahorro_dolares, cliente_antiguedad) over ventana_6 as ctrx_mcaja_ahorro_dolares_slope_6
, regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_6 as ctrx_mcuentas_saldo_slope_6
, regr_slope(mautoservicio, cliente_antiguedad) over ventana_6 as ctrx_mautoservicio_slope_6
, regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) over ventana_6 as ctrx_mtarjeta_visa_consumo_slope_6
, regr_slope(mtarjeta_master_consumo, cliente_antiguedad) over ventana_6 as ctrx_mtarjeta_master_consumo_slope_6
, regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_6 as ctrx_mprestamos_personales_slope_6
, regr_slope(mprestamos_prendarios, cliente_antiguedad) over ventana_6 as ctrx_mprestamos_prendarios_slope_6
, regr_slope(mprestamos_hipotecarios, cliente_antiguedad) over ventana_6 as ctrx_mprestamos_hipotecarios_slope_6
, regr_slope(mplazo_fijo_dolares, cliente_antiguedad) over ventana_6 as ctrx_mplazo_fijo_dolares_slope_6
, regr_slope(mplazo_fijo_pesos, cliente_antiguedad) over ventana_6 as ctrx_mplazo_fijo_pesos_slope_6
, regr_slope(minversion1_pesos, cliente_antiguedad) over ventana_6 as ctrx_minversion1_pesos_slope_6
, regr_slope(minversion1_dolares, cliente_antiguedad) over ventana_6 as ctrx_minversion1_dolares_slope_6
, regr_slope(minversion2, cliente_antiguedad) over ventana_6 as ctrx_minversion2_slope_6
, regr_slope(mpayroll, cliente_antiguedad) over ventana_6 as ctrx_mpayroll_slope_6
, regr_slope(mpayroll2, cliente_antiguedad) over ventana_6 as ctrx_mpayroll2_slope_6
, regr_slope(mcuenta_debitos_automaticos, cliente_antiguedad) over ventana_6 as ctrx_mcuenta_debitos_automaticos_slope_6
, regr_slope(mttarjeta_visa_debitos_automaticos, cliente_antiguedad) over ventana_6 as ctrx_mttarjeta_visa_debitos_automaticos_slope_6
, regr_slope(mttarjeta_master_debitos_automaticos, cliente_antiguedad) over ventana_6 as ctrx_mttarjeta_master_debitos_automaticos_slope_6
, regr_slope(mpagodeservicios, cliente_antiguedad) over ventana_6 as ctrx_mpagodeservicios_slope_6
, regr_slope(mpagomiscuentas, cliente_antiguedad) over ventana_6 as ctrx_mpagomiscuentas_slope_6
, regr_slope(mcajeros_propios_descuentos, cliente_antiguedad) over ventana_6 as ctrx_mcajeros_propios_descuentos_slope_6
, regr_slope(mtarjeta_visa_descuentos, cliente_antiguedad) over ventana_6 as ctrx_mtarjeta_visa_descuentos_slope_6
, regr_slope(mtarjeta_master_descuentos, cliente_antiguedad) over ventana_6 as ctrx_mtarjeta_master_descuentos_slope_6
, regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_6 as ctrx_mcomisiones_mantenimiento_slope_6
, regr_slope(mcomisiones_otras, cliente_antiguedad) over ventana_6 as ctrx_mcomisiones_otras_slope_6
, regr_slope(mforex_buy, cliente_antiguedad) over ventana_6 as ctrx_mforex_buy_slope_6
, regr_slope(mforex_sell, cliente_antiguedad) over ventana_6 as ctrx_mforex_sell_slope_6
, regr_slope(mtransferencias_recibidas, cliente_antiguedad) over ventana_6 as ctrx_mtransferencias_recibidas_slope_6
, regr_slope(mtransferencias_emitidas, cliente_antiguedad) over ventana_6 as ctrx_mtransferencias_emitidas_slope_6
, regr_slope(mextraccion_autoservicio, cliente_antiguedad) over ventana_6 as ctrx_mextraccion_autoservicio_slope_6
, regr_slope(mcheques_depositados, cliente_antiguedad) over ventana_6 as ctrx_mcheques_depositados_slope_6
, regr_slope(mcheques_emitidos, cliente_antiguedad) over ventana_6 as ctrx_mcheques_emitidos_slope_6
, regr_slope(mcheques_depositados_rechazados, cliente_antiguedad) over ventana_6 as ctrx_mcheques_depositados_rechazados_slope_6
, regr_slope(mcheques_emitidos_rechazados, cliente_antiguedad) over ventana_6 as ctrx_mcheques_emitidos_rechazados_slope_6
, regr_slope(matm, cliente_antiguedad) over ventana_6 as ctrx_matm_slope_6
, regr_slope(matm_other, cliente_antiguedad) over ventana_6 as ctrx_matm_other_slope_6
, regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_ctrx_quarter_slope_6
, regr_slope(Master_mfinanciacion_limite, cliente_antiguedad) over ventana_6 as ctrx_Master_mfinanciacion_limite_slope_6
, regr_slope(Master_msaldototal, cliente_antiguedad) over ventana_6 as ctrx_Master_msaldototal_slope_6
, regr_slope(Master_msaldopesos, cliente_antiguedad) over ventana_6 as ctrx_Master_msaldopesos_slope_6
, regr_slope(Master_msaldodolares, cliente_antiguedad) over ventana_6 as ctrx_Master_msaldodolares_slope_6
, regr_slope(Master_mconsumospesos, cliente_antiguedad) over ventana_6 as ctrx_Master_mconsumospesos_slope_6
, regr_slope(Master_mconsumosdolares, cliente_antiguedad) over ventana_6 as ctrx_Master_mconsumosdolares_slope_6
, regr_slope(Master_mlimitecompra, cliente_antiguedad) over ventana_6 as ctrx_Master_mlimitecompra_slope_6
, regr_slope(Master_madelantopesos, cliente_antiguedad) over ventana_6 as ctrx_Master_madelantopesos_slope_6
, regr_slope(Master_madelantodolares, cliente_antiguedad) over ventana_6 as ctrx_Master_madelantodolares_slope_6
, regr_slope(Master_mpagado, cliente_antiguedad) over ventana_6 as ctrx_Master_mpagado_slope_6
, regr_slope(Master_mpagospesos, cliente_antiguedad) over ventana_6 as ctrx_Master_mpagospesos_slope_6
, regr_slope(Master_mpagosdolares, cliente_antiguedad) over ventana_6 as ctrx_Master_mpagosdolares_slope_6
, regr_slope(Master_mconsumototal, cliente_antiguedad) over ventana_6 as ctrx_Master_mconsumototal_slope_6
, regr_slope(Master_mpagominimo, cliente_antiguedad) over ventana_6 as ctrx_Master_mpagominimo_slope_6
, regr_slope(Visa_mfinanciacion_limite, cliente_antiguedad) over ventana_6 as ctrx_Visa_mfinanciacion_limite_slope_6
, regr_slope(Visa_msaldototal, cliente_antiguedad) over ventana_6 as ctrx_Visa_msaldototal_slope_6
, regr_slope(Visa_msaldopesos, cliente_antiguedad) over ventana_6 as ctrx_Visa_msaldopesos_slope_6
, regr_slope(Visa_msaldodolares, cliente_antiguedad) over ventana_6 as ctrx_Visa_msaldodolares_slope_6
, regr_slope(Visa_mconsumospesos, cliente_antiguedad) over ventana_6 as ctrx_Visa_mconsumospesos_slope_6
, regr_slope(Visa_mconsumosdolares, cliente_antiguedad) over ventana_6 as ctrx_Visa_mconsumosdolares_slope_6
, regr_slope(Visa_mlimitecompra, cliente_antiguedad) over ventana_6 as ctrx_Visa_mlimitecompra_slope_6
, regr_slope(Visa_madelantopesos, cliente_antiguedad) over ventana_6 as ctrx_Visa_madelantopesos_slope_6
, regr_slope(Visa_madelantodolares, cliente_antiguedad) over ventana_6 as ctrx_Visa_madelantodolares_slope_6
, regr_slope(Visa_mpagado, cliente_antiguedad) over ventana_6 as ctrx_Visa_mpagado_slope_6
, regr_slope(Visa_mpagospesos, cliente_antiguedad) over ventana_6 as ctrx_Visa_mpagospesos_slope_6
, regr_slope(Visa_mpagosdolares, cliente_antiguedad) over ventana_6 as ctrx_Visa_mpagosdolares_slope_6
, regr_slope(Visa_mconsumototal, cliente_antiguedad) over ventana_6 as ctrx_Visa_mconsumototal_slope_6
, regr_slope(Visa_mpagominimo, cliente_antiguedad) over ventana_6 as ctrx_Visa_mpagominimo_slope_6
from competencia_02 
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [35]:
%%sql
copy competencia_02 to 'competencia_02.csv.gz' (FORMAT CSV, HEADER)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success
